From: https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb

Note: may not need ! in one and/or the other?

#### Only need to run install/authenticate once per Notebook (I think)

In [14]:
# !pip install earthengine-api

In [15]:
# !earthengine authenticate

In [16]:
# # Import the Earth Engine API and initialize it.
# import ee
# ee.Initialize()

# Test the earthengine command by getting help on upload.
# !earthengine upload image -h

From: GEE_downloadPRISM800_by_LandCover.py

In [1]:
import ee
ee.Initialize()
from geopandas import GeoDataFrame
from shapely.geometry import shape
# import matplotlib.pyplot as plt
import ee.mapclient
import pandas as pd

Get NLCD

In [2]:
# nlcd = ee.ImageCollection("USGS/NLCD")
# nlcd2011 = nlcd.filter(ee.Filter.eq('system:index', 'NLCD2011'))

In [3]:
# // Load a FeatureCollection of Maine Counties.
# Counties = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM').filter(ee.Filter.eq('StateName', 'Maine'))
# Counties

In [4]:
# //Imports NLCD Land Cover Data
nlcd2011 = ee.Image('USGS/NLCD/NLCD2011')

In [5]:
# Try getting points buffered and use those

In [6]:
df_coords = pd.read_csv('C:/Users/Jim/GitHub/prisons/code/R/exports/prison_ctrs_groups_latlon.csv', 
                        compression=None, header=0, sep=',', quotechar='"', low_memory=False, index_col=False)

df_coords.head()


,STATE,group,X,Y
0,AK,216,-151.529276,59.649347
1,AK,256,-135.333568,57.052658
2,AK,323,-132.377690,56.469473
3,AK,1399,-162.592879,66.896473
4,AK,2352,-152.373083,57.804570


In [54]:
# // Make a sequence the hard way.
eeList = ([[1, 2, 3, 4, 5], [1, 2, 3, 4, 5]])
# // Make a sequence the easy way!
# sequence = ee.List.sequence(1, 5);
# print('Sequence:', sequence)

eeList

[[1, 2, 3, 4, 5], [1, 2, 3, 4, 5]]

In [52]:
# // Use a method on an ee.List to extract a value.
value = eeList.get(2);
print('Value at index 2:', value);

AttributeError: 'tuple' object has no attribute 'get'

In [119]:
fc = ee.FeatureCollection([
    ee.Feature(
        ee.Geometry.Point(
            [-84.53720, 38.11316]),
        {'name': 'PrisonCtr1'}),
    ee.Feature(
        ee.Geometry.Point(
            [-84.74636, 37.71002]),
        {'name': 'PrisonCtr2'})
    ])

In [126]:
def bufferPoly(feature):
    return feature.buffer(10000) 

buffer = fc.map(bufferPoly)

buffer.getInfo()

{'type': 'FeatureCollection',
 'columns': {'name': 'String', 'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-84.5372, 38.20314865951482],
      [-84.56918403707208, 38.199562777992995],
      [-84.59861332234235, 38.18909158036699],
      [-84.62313922893195, 38.17257128887617],
      [-84.64080809967284, 38.151320485960184],
      [-84.65021718572532, 38.12703403782431],
      [-84.65062513825485, 38.10164715177824],
      [-84.64200863218798, 38.077180710743775],
      [-84.62506136762498, 38.0555803886548],
      [-84.6011364197295, 38.038562356893365],
      [-84.57213727757046, 38.027477679670824],
      [-84.54036662738707, 38.02320587990049],
      [-84.50834481736413, 38.026085803745026],
      [-84.47861190747419, 38.03588901723373],
      [-84.45352823680831, 38.05183773905489],
      [-84.43508853248947, 38.07266595918974],
      [-84.4247637392121, 38.09671912275854],
      [-84.42338296504713, 38.122084

In [9]:
# // Clip the image to the polygon geometry
nlcd2011 = nlcd2011.clip(buffer)

In [10]:
# // Extract the landcover band
landcover = nlcd2011.select('landcover')
landcover


This is cool: https://stackoverflow.com/questions/46960328/workaround-for-google-earth-engine-python-api-and-no-support-for-ee-mapclient

But this basic one works

In [14]:
ee.mapclient.centerMap(-84.74636, 37.71002, 10)
ee.mapclient.addToMap(landcover)

Follow this: https://stackoverflow.com/questions/47633088/get-results-in-an-earth-engine-python-script

In [15]:
# // Print out the frequency of landcover occurrence for each county
frequency = landcover.reduceRegions(collection = buffer, reducer = ee.Reducer.frequencyHistogram(), scale = 30)

# //Prints Feature collection
frequency = frequency.select(['histogram'])
frequency.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-84.5372, 38.20314865951482],
      [-84.56918403707208, 38.199562777992995],
      [-84.59861332234235, 38.18909158036699],
      [-84.62313922893195, 38.17257128887617],
      [-84.64080809967284, 38.151320485960184],
      [-84.65021718572532, 38.12703403782431],
      [-84.65062513825485, 38.10164715177824],
      [-84.64200863218798, 38.077180710743775],
      [-84.62506136762498, 38.0555803886548],
      [-84.6011364197295, 38.038562356893365],
      [-84.57213727757046, 38.027477679670824],
      [-84.54036662738707, 38.02320587990049],
      [-84.50834481736413, 38.026085803745026],
      [-84.47861190747419, 38.03588901723373],
      [-84.45352823680831, 38.05183773905489],
      [-84.43508853248947, 38.07266595918974],
      [-84.4247637392121, 38.09671912275854],
      [-84.42338296504713, 38.12208478169069],
      [-84.43106523278232, 38.1

#### NEXT, need to get processed to a csv


#### Trying to get a feature collection from a dataframe, see this

https://gis.stackexchange.com/questions/278607/earth-engine-convert-list-with-coordinates-and-values-into-a-feature-collection

*Need to convert df to array I believe*

In [55]:
# // get the dates
startDate = ee.Date.fromYMD(2018,1,1);
endDate = ee.Date.fromYMD(2018,2,1);

# // Create a geometry representing an export region.
myGeometry = ee.Geometry.Polygon(
[[[-61.15985870361328, -32.94904673848672],
      [-61.152305603027344, -32.944797148619905],
      [-61.15659713745117, -32.9395388987784],
      [-61.16432189941406, -32.94378874137744]]]);

In [57]:
landsatCollection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR");

# // Filter image collection to dateRange of interest
landsatDateCollection = ee.ImageCollection(landsatCollection
.filterDate(startDate,endDate));

# //Filter to AOI.
landsatAOI = ee.ImageCollection(landsatDateCollection
.filterBounds(myGeometry));

# // Filter to image with least clouds
landsatBestImage = ee.Image(landsatAOI
.sort('CLOUD_COVER')
.first());

# // Load a landsat image and select three bands.
landsatDownload = ee.Image(landsatBestImage).select(['B4', 'B5']);

In [61]:
# /////////
# //Added code that was improved after Jon's comments
# /////////


# // generate a new image containing lat/lon of the pixel and reproject it to Landsat projection
coordsImage = ee.Image.pixelLonLat().reproject(landsatDownload.projection());

joinedImage = coordsImage.addBands(landsatDownload);

valuesList = joinedImage.reduceRegion(
  reducer = ee.Reducer.toList(4),
  geometry = myGeometry
).values().get(0);

# print(valuesList);

In [65]:
valuesList.getInfo()

[[-61.15662174784661, -32.93970159517807, 474, 3731],
 [-61.15725765637795, -32.939981533850926, 446, 3776],
 [-61.15693689534622, -32.939976801649365, 455, 3774],
 [-61.15661613438958, -32.93997206862504, 508, 3684],
 [-61.156295373508, -32.93996733477794, 580, 3547],
 [-61.15789356903619, -32.940261469414814, 520, 3632],
 [-61.157572806878946, -32.94025673881013, 463, 3692],
 [-61.157252044796735, -32.94025200738267, 514, 3649],
 [-61.156931282789564, -32.94024727513243, 513, 3673],
 [-61.15661052085748, -32.940242542059416, 514, 3628],
 [-61.15628975900046, -32.94023780816362, 500, 3682],
 [-61.15820872253858, -32.94053667286187, 501, 3649],
 [-61.15788795933084, -32.94053194303132, 494, 3653],
 [-61.15756719619814, -32.940527212377965, 525, 3634],
 [-61.15724643314047, -32.94052248090183, 567, 3602],
 [-61.15692567015785, -32.94051774860292, 468, 3753],
 [-61.1566049072503, -32.940513015481216, 464, 3759],
 [-61.15628414441782, -32.94050828153672, 479, 3727],
 [-61.15596338166043, 

In [7]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]}, index=['a', 'b', 'c'])
df

,A,B
a,1,4
b,2,5
c,3,6


In [8]:
df.to_numpy()

array([[1, 4],
       [2, 5],
       [3, 6]], dtype=int64)

In [11]:
df_coords_sub1 = df_coords[0:5]

In [12]:
df_coords_sub1

,STATE,group,X,Y
0,AK,216,-151.529276,59.649347
1,AK,256,-135.333568,57.052658
2,AK,323,-132.377690,56.469473
3,AK,1399,-162.592879,66.896473
4,AK,2352,-152.373083,57.804570


In [17]:
array_sub1 = df_coords_sub1.to_numpy()
array_sub1

array([['AK', 216, -151.52927633546318, 59.64934683943813],
       ['AK', 256, -135.3335679231903, 57.052657687960405],
       ['AK', 323, -132.37769005894052, 56.46947335317286],
       ['AK', 1399, -162.59287928525558, 66.89647257681352],
       ['AK', 2352, -152.37308311926182, 57.80456975194532]], dtype=object)

In [91]:
array_sub1[1,2]

-135.3335679231903

In [92]:
valuesList = array_sub1.tolist()
valuesList

[['AK', 216, -151.52927633546318, 59.64934683943813],
 ['AK', 256, -135.3335679231903, 57.052657687960405],
 ['AK', 323, -132.37769005894052, 56.46947335317286],
 ['AK', 1399, -162.59287928525558, 66.89647257681352],
 ['AK', 2352, -152.37308311926182, 57.80456975194532]]

In [93]:
valuesList2 = ee.List(valuesList) # // Cast valuesList

In [94]:
valuesList2.getInfo()

[['AK', 216, -151.52927633546318, 59.64934683943813],
 ['AK', 256, -135.3335679231903, 57.052657687960405],
 ['AK', 323, -132.37769005894052, 56.46947335317286],
 ['AK', 1399, -162.59287928525558, 66.89647257681352],
 ['AK', 2352, -152.37308311926182, 57.80456975194532]]

In [54]:
# myFeatures = ee.FeatureCollection(valuesList.map(function(el){
#   el = ee.List(el),
#   geom = ee.Geometry.Point([ee.Number(el.get(0)), ee.Number(el.get(1))])
#   return ee.Feature(geom, {'B4':ee.Number(el.get(2)), 'B5':ee.Number(el.get(3))})
# }))

SyntaxError: invalid syntax (<ipython-input-54-9aaada3c4341>, line 2)

In [113]:
def GetCoords(el):
    el = ee.List(el)
    geom = ee.Geometry.Point([ee.Number(el.get(0)), ee.Number(el.get(1))])
    return ee.Feature(geom, {'B4':ee.String(el.get(2)), 'B5':ee.Number(el.get(3))})
    
myFeatures = ee.FeatureCollection(valuesList2.map(GetCoords))

In [114]:
myFeatures.getInfo()

EEException: GeometryConstructors.Point, argument 'coordinates': Invalid type. Expected: List<Number>. Actual: List<Object>.

In [82]:
ee.Number(ee.List(valuesList).get(1)).getInfo()

['AK', 256, -135.3335679231903, 57.052657687960405]

In [109]:
valuesList[0:2]

[['AK', 216, -151.52927633546318, 59.64934683943813],
 ['AK', 256, -135.3335679231903, 57.052657687960405]]

In [99]:
valuesList2.get(1).getInfo()

['AK', 256, -135.3335679231903, 57.052657687960405]

In [142]:
def list2fc(l):
    l1 = ee.List(l)
    geom = ee.Geometry.Point([l1.get(2), l1.get(3)])
    return ee.Feature(geom, {'state': l1.get(0), 'group': l1.get(1), 'longitude': l1.get(2), 'latitude': l1.get(3)})

In [143]:
myFeatures = ee.FeatureCollection(valuesList2.map(list2fc))

In [144]:
myFeatures

In [145]:
myFeatures.getInfo()

{'type': 'FeatureCollection',
 'columns': {'group': 'Long',
  'latitude': 'Float',
  'longitude': 'Float',
  'state': 'String',
  'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-151.52927633546318, 59.64934683943813]},
   'id': '0',
   'properties': {'group': 216,
    'latitude': 59.64934683943813,
    'longitude': -151.52927633546318,
    'state': 'AK'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-135.3335679231903, 57.052657687960405]},
   'id': '1',
   'properties': {'group': 256,
    'latitude': 57.052657687960405,
    'longitude': -135.3335679231903,
    'state': 'AK'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-132.37769005894052, 56.46947335317286]},
   'id': '2',
   'properties': {'group': 323,
    'latitude': 56.46947335317286,
    'longitude': -132.37769005894052,
    'state': 'AK'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    '

In [146]:
buffer = myFeatures.map(bufferPoly)

In [147]:
buffer

In [148]:
buffer.getInfo()

{'type': 'FeatureCollection',
 'columns': {'group': 'Long',
  'latitude': 'Float',
  'longitude': 'Float',
  'state': 'String',
  'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-151.52927633546318, 59.739335498952954],
      [-151.57914852390036, 59.73574450755719],
      [-151.62502089378356, 59.725259583524846],
      [-151.66322144900434, 59.70872125291409],
      [-151.69070474585706, 59.687453746712215],
      [-151.70529572566642, 59.663157150279055],
      [-151.70585883033257, 59.63776982959575],
      [-151.69238025038476, 59.61331278811683],
      [-151.66595934345057, 59.591728757321206],
      [-151.62871292850147, 59.57472882999115],
      [-151.58360259027856, 59.563658438916825],
      [-151.5341999939889, 59.55939266939231],
      [-151.48440852865855, 59.56226849577145],
      [-151.43816159256045, 59.57205875587029],
      [-151.3991187790859, 59.58798969124394],
      [-151.37038130208538, 59.6088